# Stacking Ensemble

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix
import warnings
!pip install kaggler
from kaggler.preprocessing import LabelEncoder
from kaggler.model import AutoLGB

     |████████████████████████████████| 829kB 5.2MB/s 
  Created wheel for kaggler: filename=Kaggler-0.9.13-cp37-cp37m-linux_x86_64.whl size=2963474 sha256=7f07e76bb0ffb19e2152f3d3213047fa92382a88a5ed07c1b2c7fe047b0e0c3a
  Stored in directory: /root/.cache/pip/wheels/5e/5a/c8/be0d5aaac1a22c4d66cc3327138818c151854d56f975a81a89
  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-cp37-none-any.whl size=7845 sha256=d1c68c03a2b8b9b8ef5ca8b1d7220209be34d7e64180dbba4ee50639e313fe3a
  Stored in directory: /root/.cache/pip/wheels/b3/61/2d/776be7b8a4f14c5db48c8e5451451cabc58dc6aa7ee3801163
Successfully built kaggler ml-metrics


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
import kaggler
print(kaggler.__version__)

0.9.13


In [5]:
pd.set_option('max_columns', 100)
warnings.simplefilter('ignore')

## Load Data

In [6]:
algo_name = 'esb'
model_name = f'{algo_name}'

predict_val_file = f'{model_name}.val.txt'
predict_tst_file = f'{model_name}.tst.txt'
submission_file = f'{model_name}.sub.csv'

index_col = 'index'
target_col = 'credit'

In [7]:
trn = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/credit_data/train.csv', index_col=index_col)
print(trn.shape)
trn.head()

(26457, 19)


,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,,
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [8]:
tst = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/credit_data/test.csv', index_col=index_col)
print(tst.shape)
tst.head()

(10000, 18)


,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
index,,,,,,,,,,,,,,,,,,
26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0
26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0
26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0
26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0
26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0


In [9]:
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/credit_data/sample_submission.csv', index_col=index_col)
print(sub.shape)
sub.head()

(10000, 3)


,0,1,2
index,,,
26457,0,0,0
26458,0,0,0
26459,0,0,0
26460,0,0,0
26461,0,0,0


## Feature Engineering

### Label Encoding for Categorical Features

범주형 변수를 레이블 인코딩합니다. `kaggler` 패키지의 `LabelEncoder`를 사용하면 희귀 값들 (아래 코드에서는 10번 미만 등장한 값들)을 새로운 하나의 범주로 그룹지어주고, 결측값도 새로운 범주로 간주합니다.

In [10]:
def days_to_year(x):
    return (x*-1)/365

def minus(x):
    return x * -1

def remove_outlier(trn, column):
    
    df = trn[column]
    
    # 1분위수
    quan_25 = np.percentile(df.values, 25)
    
    # 3분위수
    quan_75 = np.percentile(df.values, 75)
    
    iqr = quan_75 - quan_25
    
    lowest = quan_25 - iqr * 5
    highest = quan_75 + iqr * 5
    outlier_index = df[(df < lowest) | (df > highest)].index
    print('outlier의 수 : ' , len(outlier_index))
    print(df.iloc[outlier_index])
    trn.drop(outlier_index, axis = 0, inplace = True)
    
    return trn

candidate = ['child_num']
for cand in candidate:  
    trn = remove_outlier(trn,cand)

trn.reset_index(drop = True,inplace = True)
len(trn)

def add_var(data):
    
    # 개개인을 구분할 수 있는 변수들을 묶어서 생성
    data['personal_id'] = data['gender'] + "_" + data['DAYS_BIRTH'].astype(str) + "_" + data['income_total'].astype(str) + "_" + data['income_type'].astype(str)
    
    # 카드를 생성한 기간도 같은 경우가 있어서 begin을 추가하여 하나의 변수를 더 생성
    data['personal_begin_id'] = data['gender'] + "_" + data['DAYS_BIRTH'].astype(str) + "_" + data['income_total'].astype(str) + "_" + data['income_type'].astype(str) + "_" + data['begin_month'].astype(str)
    
    # 그외의 변수들을 조합하여 하나의 변수로 추가 생성
    data['g_r_c'] = data['gender'] + "_" + data['reality'] + "_" + data['car'] 
    data['p_w_e'] = data['phone'].astype(str) + "_" + data['work_phone'].astype(str) + "_" + data['email'].astype(str) 
    
    return data

trn = add_var(trn)
tst = add_var(tst)

def numeric_process(data):
    
    # income_total 변수 전처리
    # 만단위로 생성
    data['income_total'] = data['income_total']/10000
    # 편차 제곱 변수 생성
    data['income_total_dev'] = (data['income_total'] - data['income_total'].mean())**2
    # 로그 변환
    data['income_total_log'] = data['income_total'].apply(np.log1p)

    # DAYS_EMPLOYED 변수 전처리
    # 0 이상인 경우 0으로 모두 변환
    data.loc[data['DAYS_EMPLOYED'] >= 0,'DAYS_EMPLOYED']=0
    # day를 year로 변환
    data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].apply(days_to_year) 
    # 로그 변환
    data['DAYS_EMPLOYED_log'] = data['DAYS_EMPLOYED'].apply(np.log1p)

    # begin_month 변수 전처리
    # 마이너스 변환
    data['begin_month'] = data['begin_month'].apply(minus)
    
    # DAYS_BIRTH 변수 전처리
    # day를 year로 변환
    data['DAYS_BIRTH'] = data['DAYS_BIRTH'].apply(days_to_year)

    #  DAYS_BIRTH, DAYS_EMPLOYED, income_total변수를 조합하여 RATIO 변수 생성
    data['EMPLOYED_BIRTH_RATIO'] = data['DAYS_EMPLOYED']/data['DAYS_BIRTH']
    data['INCOME_EMPLOYED_RATIO'] = data['income_total']/data['DAYS_EMPLOYED']
    data['INCOME_BIRTH_RATIO'] = data['income_total']/data['DAYS_BIRTH']
    
    # 가족수 - 자식수
    data['diff_fam_child'] = data['family_size'] - data['child_num']
    # chid_num과 family_size는 다음과 같이 최대 2와 5가 되도록 전처리
    data.loc[data['child_num'] >= 2,'child_num'] = 2
    data.loc[data['family_size'] >= 5,'child_num'] = 5
    # 가족수와 자녀수 sum 변수 추가
    data['FAM_CHILD_SUM'] = data[['child_num', 'family_size']].sum(axis=1)
    
    # income을 가족 수 및 자식 수로 나눈 비율
    data['INCOME_FAM_RATIO'] = data['income_total']/data['family_size']
    data['INCOME_child_num_RATIO'] = data['income_total']/data['child_num']
    
    # 일을하게 된 시점 변수 추가
    data['BIRTH_MINUS_EMPLOYED'] = data['DAYS_BIRTH'] - data['DAYS_EMPLOYED']
    # income total 변수에 before_EMPLOYED로 나눈 변수 추가
    data['INCOME_BIRTH_MINUS_EMPLOYED_RATIO'] = data['income_total']/data['BIRTH_MINUS_EMPLOYED']
    
    return data 
  


trn = numeric_process(trn)
tst = numeric_process(tst)

def occype_process(data):
    
    # occyp_type 변수에만 있는 결측치를 'NAN' 값으로 대체
    data['occyp_type'] = data['occyp_type'].fillna('NAN')
    # 경력이 없고 직업군이 none인 사람은 no_work로 대체
    data.loc[(data['DAYS_EMPLOYED'] == 0) & (data['occyp_type'] == 'NAN'), 'occyp_type'] = 'no_work'
    print(data['occyp_type'].value_counts(), '\n\n')

    return data

trn = occype_process(trn)
tst = occype_process(tst)

def make_bin(df, variable, n):
    
    data = df
    count, bin_dividers = np.histogram(data[variable], bins=n)
    bin_names=[str(i) for i in range(n)]
    data['%s_bin' % variable] = pd.cut(x=data[variable], bins=bin_dividers, labels=bin_names, include_lowest=True)
    data['%s_bin' % variable] = pd.factorize(data['%s_bin' % variable])[0]
    print(data['%s_bin' % variable], '\n\n')
    
    return data

trn = make_bin(trn, 'DAYS_BIRTH', n=10)
tst = make_bin(tst, 'DAYS_BIRTH', n=10)

trn = trn.drop(['income_total', 'DAYS_EMPLOYED', 'FLAG_MOBIL'], axis=1)
tst = tst.drop(['income_total', 'DAYS_EMPLOYED', 'FLAG_MOBIL'], axis=1)



outlier의 수 :  6
index
8462     14
9021     14
10731    19
25313     7
25390    14
25638     7
Name: child_num, dtype: int64
Laborers                 4512
no_work                  4438
NAN                      3733
Core staff               2646
Sales staff              2539
Managers                 2167
Drivers                  1572
High skill tech staff    1040
Accountants               902
Medicine staff            864
Cooking staff             457
Security staff            424
Cleaning staff            401
Private service staff     243
Low-skill Laborers        127
Waiters/barmen staff      123
Secretaries                97
Realty agents              63
HR staff                   62
IT staff                   41
Name: occyp_type, dtype: int64 


Laborers                 1699
no_work                  1697
NAN                      1455
Sales staff               946
Core staff                945
Managers                  845
Drivers                   563
Medicine staff            343
Hi

In [11]:
cat_cols = [x for x in trn.columns if trn[x].dtype == 'object']
num_cols = [x for x in trn.columns if x not in cat_cols + [target_col]]
feature_cols = num_cols + cat_cols
print(len(feature_cols), len(cat_cols), len(num_cols))

lbe = LabelEncoder(min_obs=10)
trn[cat_cols] = lbe.fit_transform(trn[cat_cols])
tst[cat_cols] = lbe.transform(tst[cat_cols])

32 12 20


## Level-1 Base Model Training

In [12]:
n_est = 1000
seed = 42
n_fold = 5
n_class = 3

lgb_params = {
    'metric': 'multi_logloss',
    'n_estimators': n_est,
    'objective': 'multiclass',
    'random_state': seed,
    'learning_rate': 0.03,
    'min_child_samples': 20,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
    'num_leaves': 63,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'num_class': n_class
}

xgb_params = {
     'booster' : 'gbtree',
    'tree_method' : 'gpu_hist',
    'predictor' : 'gpu_predictor',    
    'objective' : 'multi:softprob',
    'eval_metric' : 'mlogloss',
    'n_estimators' : 5000,
    'max_depth' : 9,
    'min_child_weight' : 5,    
    'learning_rate' : 0.012727,    
    'subsample' : 0.91020,
    'colsample_bytree' : 0.77959,    
    'colsample_bylevel' : 0.64898,
    'lambda' : 0.05,
    'alpha' : 1,    
    'seed' : 2018
}

rf_params = {
    'max_depth': 15,
    'min_samples_leaf': 8,
    'random_state': seed
}

In [13]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


base_models = {'rf': RandomForestClassifier(**rf_params), 
               'lgb': LGBMClassifier(**lgb_params),
               'xgb': XGBClassifier(),
               'et': ExtraTreesClassifier(bootstrap=True, 
                                          criterion='entropy', 
                                          max_features=0.55, 
                                          min_samples_leaf=8, 
                                          min_samples_split=4, 
                                          n_estimators=100)}

In [14]:
trn= trn.replace([np.inf,-np.inf], 1)
tst = tst.replace([np.inf,-np.inf], 1)

#pd.DataFrame(trn).fillna()
#pd.DataFrame(tst).fillna()

#np.nan_to_num(trn)
# object label encoding -> inf 값 발생, 데이터 시각화 다시하고 적절한 대체값 설정

In [15]:
from copy import copy
feature_cols =[x for x in trn.columns if x not in [target_col]]

X = trn[feature_cols]
y = trn[target_col]
X_tst = tst[feature_cols]

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

p_dict = {}
p_tst_dict = {}
for name in base_models:
    print(f'Training {name}:')
    p = np.zeros((X.shape[0], n_class), dtype=float)
    p_tst = np.zeros((X_tst.shape[0], n_class), dtype=float)
    for i, (i_trn, i_val) in enumerate(cv.split(X, y)):
        clf = copy(base_models[name])
        clf.fit(X.iloc[i_trn], y[i_trn])

        p[i_val] = clf.predict_proba(X.iloc[i_val])
        p_tst += clf.predict_proba(X_tst) / n_fold

    p_dict[name] = p
    p_tst_dict[name] = p_tst
    print(f'\tCV Log Loss: {log_loss(y, p):.6f}')

Training rf:
	CV Log Loss: 0.748741
Training lgb:
	CV Log Loss: 0.715739
Training xgb:
	CV Log Loss: 0.792646
Training et:
	CV Log Loss: 0.755307


## Level-2 Stacking

In [16]:
X = pd.DataFrame(np.hstack([x for _, x in p_dict.items()]))
X_tst = pd.DataFrame(np.hstack([x for _, x in p_tst_dict.items()]))

p = np.zeros((X.shape[0], n_class), dtype=float)
p_tst = np.zeros((X_tst.shape[0], n_class), dtype=float)
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y)):
    if i_cv == 0:
        clf = AutoLGB(objective='multiclass', metric='multi_logloss', params={'num_class': n_class}, 
                      feature_selection=False, n_est=10000)
        clf.tune(X.iloc[i_trn], y[i_trn])
        n_best = clf.n_best
        features = clf.features
        params = clf.params
        print(f'best iteration: {n_best}')
        print(f'selected features ({len(features)}): {features}')        
        pprint(params)
        clf.fit(X.iloc[i_trn], y[i_trn])
    else:
        train_data = lgb.Dataset(X[features].iloc[i_trn], label=y[i_trn])
        clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    p[i_val] = clf.predict(X[features].iloc[i_val])
    p_tst += clf.predict(X_tst[features]) / n_fold

100%|██████████| 100/100 [01:29<00:00,  1.12it/s, best loss: 0.7233786505114463]
best iteration: 65
selected features (12): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
{'bagging_fraction': 0.5,
 'bagging_freq': 1,
 'boosting': 'gbdt',
 'feature_fraction': 0.5,
 'feature_pre_filter': False,
 'lambda_l1': 0.1,
 'lambda_l2': 0.1,
 'learning_rate': 0.08068382668913798,
 'max_depth': 4,
 'metric': 'multi_logloss',
 'min_child_samples': 100,
 'num_class': 3,
 'num_leaves': 63,
 'num_threads': -1,
 'objective': 'multiclass',
 'seed': 42,
 'verbosity': -1}


In [17]:
print(f'CV Log Loss: {log_loss(y, p):.6f}')
np.savetxt(predict_val_file, p, fmt='%.6f')
np.savetxt(predict_tst_file, p_tst, fmt='%.6f')

CV Log Loss: 0.703356


## Save the Submission File

In [18]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2
index,,,
26457,0.076037,0.140176,0.783787
26458,0.262622,0.196363,0.541015
26459,0.064346,0.086272,0.849382
26460,0.104265,0.105581,0.790155
26461,0.099300,0.134355,0.766345


In [19]:
sub.to_csv(submission_file)

In [20]:
submission_file

'esb.sub.csv'